# Question-Answering
Question answering is becoming more and more popular, as people seem to prefer to get answer interactively rather than reading long documents, and information can be found more quickly. 

We will the SubjQA dataset, which contains over 10'000 customer reviews in English  in six domains. 

**Note:** If you get a `NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.` when running the cell `print(subjqa["train"]["answers"][1])`: uncomment the following two `!pip` commands, restart the kernel, and try again.

In [ ]:
# !pip install -U typing-extensions
# !pip install fsspec==2023.9.2

## Preparations

In [ ]:
from datasets import get_dataset_config_names

The `subjqa` has reviews from different domains:

In [ ]:
domains = get_dataset_config_names("subjqa", trust_remote_code=True)
domains

We will use the feedbacks on electronics:

In [ ]:
from datasets import load_dataset

subjqa = load_dataset("subjqa", name="electronics")

Let's first look at some examples:

In [ ]:
print(subjqa["train"]["answers"][1])

We re-organize the data into a structure that is more easy to handle:

In [ ]:
import pandas as pd

dfs = {split: dset.to_pandas() for split, dset in subjqa.flatten().items()}

And we want to get an overview over how many data points we have:

In [ ]:
for split, df in dfs.items():
    print(f"Number of questions in {split}: {df['id'].nunique()}")

Here are some of the most relevant attributes:

In [ ]:
qa_cols = ["title", "question", "answers.text",
           "answers.answer_start", "context"]
sample_df = dfs["train"][qa_cols].sample(2, random_state=7)
sample_df

In [ ]:
sample_df.loc[791, 'context']

In [ ]:
sample_df.loc[1159, 'context']

**Exercise:** Now it is your turn. Inspect at least 2 sample rows in more detail to get a better feeling for the data.

In [ ]:
your_sample_df = dfs["train"][qa_cols].sample(2, random_state=10) # fill in this line two sample two rows
your_sample_df.reset_index(inplace = True)

<details>
  <summary>Click to see the solution</summary>

  ```python
  # solution
  your_sample_df = dfs["train"][qa_cols].sample(2, random_state=10)
  your_sample_df.reset_index(inplace = True)
  ```
</details>

In [ ]:
print(your_sample_df.loc[0, ...]) # extract the column "context" from the first row
print(your_sample_df.loc[0, ...]) # extract the column "question" from the first row
print(your_sample_df.loc[0, ...]) # extract the column "answers.text" from the first row

<details>
  <summary>Click to see the solution</summary>

  ```python
# solution
print(your_sample_df.loc[0, "context"])
print(your_sample_df.loc[0, "question"])
print(your_sample_df.loc[0, "answers.text"])
  ```
</details>

In [ ]:
print(your_sample_df.loc[..., ...]) # extract the column "context" from the second row
print(your_sample_df.loc[..., ...]) # extract the column "question" from the second row
print(your_sample_df.loc[..., ...]) # extract the column "answers.text" from the second row

<details>
  <summary>Click to see the solution</summary>

  ```python
# solution
print(your_sample_df.loc[1, "context"])
print(your_sample_df.loc[1, "question"])
print(your_sample_df.loc[1, "answers.text"])
  ```
</details>

As mentioned in the slides, we see that some of the answers are grammatically not correct sentences, and some cannot be answered (such as "how is the battery?")

## Span Classification
A popular approach is to define a task to find the start and end token of the context that is contains the answer.

Many models for this are available on e.g., Huggingface Hub. We choose a small one by deepset, a German AI start-up

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
question = "How much music can this hold?"
context = """An MP3 is about 1 MB/minute, so about 6000 hours depending on file size."""
inputs = tokenizer(question, context, return_tensors="pt")

As before, we can get the tokens that the tokenizer has generated - and with `tokenizer.decode`, we see how our question and context are represented:

In [ ]:
inputs

In [ ]:
print(tokenizer.decode(inputs["input_ids"][0]))

After tokenization, we can use the model to predict the start and end token:

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

s_scores = start_logits.detach().numpy().flatten()
e_scores = end_logits.detach().numpy().flatten()
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
token_ids = range(len(tokens))

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
colors = ["C0" if s != np.max(s_scores) else "C1" for s in s_scores]
ax1.bar(x=token_ids, height=s_scores, color=colors)
ax1.set_ylabel("Start Scores")
colors = ["C0" if s != np.max(e_scores) else "C1" for s in e_scores]
ax2.bar(x=token_ids, height=e_scores, color=colors)
ax2.set_ylabel("End Scores")
plt.xticks(token_ids, tokens, rotation="vertical")
plt.show()

Putting the components together, we can now get the result:

In [ ]:
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]
answer = tokenizer.decode(answer_span)
print(f"Question: {question}")
print(f"Answer: {answer}")

**Exercise:** Now it is your turn. Provide a context and question of your choice. Use the model to answer that question.

In [ ]:
your_question = ... # fill in this line
your_context = ... # fill in this line

your_inputs = tokenizer(..., ..., return_tensors="pt") # fill in this line

with torch.no_grad():
  outputs = model(...) # fill in this line

start_logits = ... # fill in this line
end_logits = ... # fill in this line
start_idx = ...# fill in this line
end_idx = ...# fill in this line
answer_span = ...# fill in this line
answer = ...# fill in this line

print(f"Question: {your_question}")
print(f"Answer: {answer}")

<details>
  <summary>Click to see the solution</summary>

  ```python
# solution
your_question = "How many servings does this recipe make?" # possible solution
your_context = """
This recipe for spaghetti bolognese uses 500 grams of pasta and 1 kilogram of vegan beef,
along with other ingredients. It is designed to serve 4 people, with each serving
being approximately 125 grams of pasta and 250 grams of vegan beef.
""" # possible solution
your_inputs = tokenizer(your_question, your_context, return_tensors="pt")

with torch.no_grad():
  outputs = model(**your_inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits) + 1
answer_span = your_inputs["input_ids"][0][start_idx:end_idx]
answer = tokenizer.decode(answer_span)

print(f"Question: {your_question}")
print(f"Answer: {answer}")
  ```
</details>